Now, we turn to elliptic equations. Elliptic equations do not have a "time-like" variable, in the sense that the solution at every point in the domain depends on all of the boundary conditions. Picking one of the variables to designate as "time," setting up a finite difference scheme with initial conditions and boundary conditions, and stepping through "time" is unlikely to work for elliptic equations since the solution at any point depends on all of the boundary conditions, including the "future" from the perspective of our designated variable. Thus, such a scheme could never converge, as it does not depend on all of the information which the true solution depends on.

 One interpretation of elliptic equations is as steady-state versions of parabolic equations. In other words, the solution to an elliptic equation describes the equilibrium state of a system, rather than how it evolves over time. As an example, recall the forced heat equation:
$$
    u_t + \Delta u = f.
$$
Along with some initial values and boundary conditions, we can model the evolution of the system over time. It is natural to wonder if we can model what happens as time goes to infinity. Since this is a dissipative system, it is reasonable to expect that it approaches some limiting solution as time goes to infinity (contrast this to the wave equation with reflective boundary conditions, in which we would not expect things to converge as time goes to infinity). One way to impose the requirement that eventually, the solution does not change in time is to specify that $u_t = 0$. If we do this, we get the equation
$$
    \Delta u = f.
$$
This is Poisson's equation, and we have heuristically justified the interpretation of it as giving the limiting or steady-state solution to the heat equation. 

Suppose a function $u$ is the steady-state solution to some time-dependent PDE. One way to find $u$ is to begin with arbitrary initial conditions and then let it evolve until the solution changes so slowly in time that we regard it as essentially constant. This limit may be a good approximation to $u$. Numerically, this translates to iterating a numerical scheme for the time-dependent PDE, with constant boundary conditions, until the solution stabilizes.

If $U^n$ is the $n$-th iteration of the scheme, we can measure whether the solution is changing in essentially two ways: comparing $U^n$ to $U^{n+1}$ or comparing $U^n$ to $U^m$ for some $m$ many time steps in the future. The second option has more strength in detecting convergence. As a toy example, consider the series $\sum 1/k$, which diverges. Nonetheless, the term-to-term differences in the sum are just $1/k$, which goes to $0$ fairly quickly. In contrast, $\sum_{k=1}^m 1/k - \sum_{k=1}^n 1/k \approx \log(m-n)$, and we have a better chance of figuring out that we aren't getting convergence than when just looking at $1/k$. 

While the method of iterating a time-dependent version of the elliptic PDE has the advantage of applying previous knowledge to a new problem, there are better methods, such as the finite element method. For clarity, we start by developing finite element method in one dimension before extending it to higher dimensions. The one-dimensional Poisson equation  on $[0, 1]$ with forcing term $f$ and homogeneous Dirichlet boundary conditions is
\begin{cases}
    -u''(x) = f(x), x \in (0, 1),\\
    u(0) = u(1) = 0.
\end{cases}
This is just an ODE, but is not an  initial value problem due to the specification of the value of $u(1)$. If, in contrast, the boundary conditions specified $u(0)$ and $u'(0)$, we could solve this typical numerical methods for initial value problems such as Runge-Kutta.

There are two main ideas for the finite element method. The first is that we want to approximate $u$ by some function living in a finite-dimensional vector space. The second is to transform the problem into a "variational" problem on this finite-dimensional space.

For the variational problem, we define a vector space of functions in which we hope the solution lives. So we look for functions from $[0, 1]$ to $\mathbb R$ in the (infinite-dimensional) vector space
$$
    V = \{v: v(x) \text{ is continuous },
    v'(x) \text{ is piecewise continuous and bounded}, v(0) = v(1) = 0\}.
$$

As a remark, this is not the standard space for finite element methods, which we will introduce later, but suffices for now. If there is a nice solution to the PDE, then it should live in this space. So suppose $u$ is the solution to the one-dimensional Poisson equation above, and let $v$ be an arbitrary function in $V$. We define the *inner product* by
$$
    (u, v) = \int_0^1 u(x) v(x) \, dx.
$$
The inner product is symmetric: $(u, v) = (v, u)$. It is also linear in each entry: $(u, \alpha v+w) = \alpha(u, v) + (u, w)$, and similarly for the second entry. The inner product can be thought of as an extension of the dot product in finite-dimensional vector spaces. Using integration by parts, 
$$
    (f, v) = (-u'', v) = \int_0^1 -u''(x) v(x) \, dx = -[u'(x) v(x)]^1_0 + \int_0^1 u'(x) v'(x) \, dx.
$$
Since $v(0) = v(1) = 0$ by the definition of $V$, the term $-[u'(x) v(x)]^1_0$ is just $0$ and we get the identity that $(f, v) = (u', v')$ for any $v \in V$.

Thus, if $u$ is any solution to the PDE which is sufficiently smooth that integration by parts can apply, we must have $(f, v) = (u', v')$ for all $v \in V$. On the other hand, suppose that $u''$ exists and is continuous and that $(f, v) = (u', v')$ for all $v \in V$. Then by a similar integration by parts argument, $(-u'', v) = (u', v')$, and we assumed $(u', v') = (f, v)$, so we see that $(-u'' - f, v) = 0$ for any $v \in V$. Since $-u''-f$ is continuous, if it is not $0$ everywhere, then there is some point $x_0$ at which it is nonzero and some neighborhood $I = (x_0-\delta, x_0+\delta)$ in which it is nonzero. Take a function $v$ which is positive on $I$ and $0$ elsewhere (such a function [exists](https://en.wikipedia.org/wiki/Bump_function)). Then $(-u''-f, v) \neq 0$, which is a contradiction. So we have shown that, under the assumption that $u$ and $f$ are continuous, any function $u$ satisfying $(f, v) = (u', v')$ for all $v \in V$ must also satisfy $-u'' - f = 0$, i.e., $-u'' = f$, which is our PDE.

This motivates the definition of the variational problem:

Find some $u \in V$ such that $(u', v') = (f, v)$ for all $v \in V$.

It's worth noting that if a solution exists, it must be unique: if $u$ and $\tilde u$ both satisfy the variational problem, then $(u' - \tilde u', v') = 0$ for all $v \in V$, telling us that $u' = \tilde u'$, and so $u$ and $\tilde u$ differ by a constant. But they both satisfy the boundary condition, so they must be equal.

We can find an additional reformulation of the problem in terms of an energy functional. For $v \in V$, we define the *total energy* of as 
$$
    F(v) = \frac 1 2 (v', v') - (f, v).
$$
This is motivated by the fact that the true solution minimizes this energy functional. To see this, let $v$ be an arbitrary element of $V$. We want to show that $F(u) \leq F(v)$. Define $w = v-u$, so that $v = u + w$. Using the linearity and symmetry of the inner product, we compute
\begin{align*}
    F(v) = F(u+w) &= (u'+w', u'+w')/2 - (f, u+w)\\
        &= (u', u')/2 + (w', w')/2 + (u', w') - (f, u) - (f, w)\\
        &= (w', w')/2 + (u', u')/2 - (f, u) + [(u', w') - (f, w)]\\
        &= (w', w')/2 + F(u)\\
        \geq F(u),
\end{align*}
where we have used the definition of $F(u)$, the fact that $(u', w') - (f, w) = 0$ from the variational problem, and the fact that $(w', w') = \int_0^1 (w')^2 \, dx$.

Still, $V$ is an infinite dimensional vector space, so we cannot hope to even represent an arbitrary element of $V$. We can get a numerical scheme by working with a finite-dimensional subspace of $V$. 